In [95]:
#lots of physics formulas calculated to be input here
def compare(x10,y10,vx1,vy1,x20,y20,vx2,vy2):#, minzone, maxzone):
    if vx1*vy2!=vx2*vy1:#if not parallel
        numer, denom = (vx2*(y10-y20) + vy2*(x20-x10)), (vx1*vy2-vx2*vy1)
        x_coll, y_coll = x10 + numer*vx1/denom, y10 + numer*vy1 / denom
        if x_coll>=minzone and x_coll<=maxzone and y_coll>=minzone and y_coll<=maxzone and numer/denom>=0 and (vx1*(y10-y20)+vy1*(x20-x10))/denom>=0: # last two are t_coll1, t_coll2
            return True
    elif (y10-y20)*vx1*vx2==vx2*vy1*x10-vx1*vy2*x20:
        raise ValueError("Oh, no! There are parallel, but coinciding, straight lines!")
    return False

with open("input24.txt", "r") as f:
    data = f.read().split("\n")
# data = [l.split(" @ ") for l in lines]
data = [list(map(int,d.replace(" @ ",",").split(","))) for d in data]

# minzone, maxzone = 7, 27
minzone, maxzone = 200000000000000, 400000000000000
count = 0 
for i in range(len(data)):
    x10, y10 = data[i][0:2]
    vx1, vy1 = data[i][3:5]
    for line2 in data[i+1:]:
        x20, y20 = line2[0:2]
        vx2, vy2 = line2[3:5]
        count+=compare(x10,y10,vx1,vy1,x20,y20,vx2,vy2)
count

20434

## Part 2

If any 2 lines were coplanar, it would be much simpler, as is the case for the example input (shown below).

Since this is not the case, let's try by picking 2 lines and trying different collision times with these two. Two such collisions specify the full rock trajectory

In [52]:
#ok, this is not great, but it's a decent solution, good time (~15s), and feels mine
import numpy as np

def find_candidates(data, i=0, j=1):
    def det2int(m): #with the ints to ensure good representation, numbers are too high!
        return (int(m[0,0])*int(m[1,1]) -int(m[0,1])*int(m[1,0]))
    r10, v1 = np.array(data[i][:3]), np.array(data[i][3:])
    r20, v2 = np.array(data[j][:3]), np.array(data[j][3:])
    listvx, listvy = {d[3] for d in data}, {d[4] for d in data}
    minvx, maxvx, minvy, maxvy = min(listvx), max(listvx), min(listvy), max(listvy)

    #Trying initial velocities vpx, vpy, which are integers
    candidates = set()
    for vrx in range(minvx, maxvx+1):
        for vry in range(minvy, maxvy+1):
        #If it collides with hailstones 1 and 2, we know the initial conditions. We get a system for x_p0, y_p0 whose relevant matrices are
            m = np.array([[v1[1]-vry, -(v1[0]-vrx)], [v2[1]-vry, -(v2[0]-vrx)]], dtype=object)
            ind_col = np.array([r10[0]*(v1[1]-vry) - r10[1]*(v1[0]-vrx), r20[0]*(v2[1]-vry) - r20[1]*(v2[0]-vrx)], dtype=object)
            m_x, m_y = m.copy(), m.copy()
            m_x[:,0], m_y[:,1]  = ind_col, ind_col
            detm, detmx, detmy = det2int(m), det2int(m_x), det2int(m_y)
            if detm==0:# hoping for solution to be well-behaved! #and (detmx!=0 or detmy!=0):
                continue
            x_p0, y_p0 = detmx/detm, detmy/detm
            if x_p0%1==0 and y_p0%1==0:
                x_p0, y_p0 = int(x_p0), int(y_p0)
                #for z_p0, vrz,
                t1, t2 = ((x_p0-r10[0])/(v1[0]-vrx)), ((x_p0-r20[0])/(v2[0]-vrx))
                if np.isnan(t1):
                    t1 = ((y_p0-r10[1])/(v1[1]-vry))
                if np.isnan(t2):
                    t2 = ((y_p0-r20[1])/(v2[1]-vry))
                if np.isnan(t1) or np.isnan(t2): 
                    # continue # this would be hoping for solution to be well-behaved
                    raise ValueError("t1, t2 are nan")
                #relevant matrices
                m = np.array([[1, t1],[1,t2]], dtype=object)
                ind_col = np.array([r10[2] + t1*v1[2], r20[2] + t2*v2[2]], dtype=object)
                m_z, m_vz = m.copy(), m.copy()
                m_z[:,0], m_vz[:,1] = ind_col, ind_col
                z_p0, vrz = int(det2int(m_z))/int(det2int(m)), det2int(m_vz)/det2int(m)
                if z_p0%1==0 and vrz%1==0:#if all initial conditions are integers, it's a candidate!
                    candidates.add((int(x_p0), int(y_p0), int(z_p0), int(vrx), int(vry), int(vrz), int(t1), int(t2)))
    return candidates

def analyze_candidates(data, candidates, k=2):
    r30, v3 = np.array(data[k][:3]), np.array(data[k][3:])
    for x_p0, y_p0, z_p0, vrx, vry, vrz, t1, t2 in candidates:
        if (x_p0-r30[0])*(vry-v3[1]) == (y_p0-r30[1])*(vrx-v3[0]) and (y_p0-r30[1])*(vrz-v3[2]) == (z_p0-r30[2])*(vry-v3[1]): #checking for collision with third hailstone
            print("Found: initial position", x_p0, y_p0, z_p0,"\b, velocity", vrx, vry, vrz,"\b, collision times" ,t1, t2, "sum (output):", x_p0+y_p0+z_p0)
    return None

with open("input24.txt", "r") as f:
    data = [list(map(int,d.replace(" @ ",",").split(","))) for d in f.read().split("\n")]

#using three random hailstones
i,j,k = random.randint(0,len(data)), random.randint(0,len(data)), random.randint(0,len(data))
while i==j or j==k or k==i:
    i,j,k = random.randint(0,len(data)), random.randint(0,len(data)), random.randint(0,len(data))
print("Using hailstones",i,j,k)
candidates = find_candidates(data, i=i, j=j)
print("Analyzing",len(candidates),"candidates")
analyze_candidates(data, candidates, k=k)

Using hailstones 9 14 54


C:\Users\Márcio\AppData\Local\Temp\ipykernel_13792\3283295897.py:28: RuntimeWarning: invalid value encountered in longlong_scalars
  t1, t2 = ((x_p0-r10[0])/(v1[0]-vrx)), ((x_p0-r20[0])/(v2[0]-vrx))


Analyzing 70 candidates
Found: initial position 344525619959965 437880958119624 242720827369528, velocity -99 -269 81, collision times 833147003717 646524273976 sum (output): 1025127405449117


In [7]:
#This is fucking beautiful. I wish I had done it
# Author (except for my few tweaks): mgtezak https://github.com/mgtezak/Advent_of_Code/blob/master/2023/Day_24.py
import sympy as sp
import random

with open("input24.txt", "r") as f:
    data = f.read().split('\n')

#using three random hailstones
i,j,k = random.randint(0,len(data)), random.randint(0,len(data)), random.randint(0,len(data))
while i==j or j==k or k==i:
    i,j,k = random.randint(0,len(data)), random.randint(0,len(data)), random.randint(0,len(data))
    
three_hailstones = []
for n in [i,j,k]:
    nums = data[n].replace('@', ',').split(',')
    three_hailstones.append(tuple(map(int, nums)))

unknowns = sp.symbols('x0 y0 z0 vx vy vz t1 t2 t3')
x0, y0, z0, vx, vy, vz, *coll_times = unknowns

equations = []  # build system of 9 equations with 9 unknowns
for t, pos_vel in zip(coll_times, three_hailstones):
    equations.append(sp.Eq(x0 + t*vx, pos_vel[0] + t*pos_vel[3]))
    equations.append(sp.Eq(y0 + t*vy, pos_vel[1] + t*pos_vel[4]))
    equations.append(sp.Eq(z0 + t*vz, pos_vel[2] + t*pos_vel[5]))

sol = sp.solve(equations, unknowns).pop()

print("Final output:", sum(sol[:3]),"\nFull solution data (x0 y0 z0 vx vy vz t1 t2 t3):", sol, "\nRows used:",i,j,k)

Final output: 1025127405449117 
Full solution data (x0 y0 z0 vx vy vz t1 t2 t3): (344525619959965, 437880958119624, 242720827369528, -99, -269, 81, 516434301805, 692825718608, 192080506605) 
Rows used: 0 74 146


In [3]:
#works for the example, but not for the actual input. In principle works for the input, but the times are just too long to be searched ~10**11. 
# It was a good catch that the velocities are smaller, hence more searchable
import numpy as np
with open("exampleinput24.txt", "r") as f:
    data = [list(map(int,d.replace(" @ ",",").split(","))) for d in f.read().split("\n")]

r10, v1 = np.array(data[0][:3]), np.array(data[0][3:])
r20, v2 = np.array(data[1][:3]), np.array(data[1][3:])

#Trying initial times t1, t2. Assuming they're integers, but they might not be...
for added in range(10):
    for t1 in range(0, added+1):
        t2 = added - t1
        if t2 == t1:
            continue
        #If it collides with hailstone 1 at t1 and with 2 at t2, the rock will have initial position and velocity
        r_p0  = -(r20*t1 - r10*t2 +(v2-v1)*t1*t2)/(t2-t1)
        if not (r_p0%1).any():
            v_r = (r20 - r10 + v2*t2 - v1*t1)/(t2-t1)
            if not (v_r%1).any():
                r_p0, v_r = r_p0.astype(np.int64), v_r.astype(np.int64)
                #test if it collides with the next hailstone
                for row in data[2:]:
                    r30, v3 = np.array(row[:3]), np.array(row[3:])
                    if (r_p0-r30)[0]*(v_r-v3)[1] != (r_p0-r30)[1]*(v_r-v3)[0] or (r_p0-r30)[1]*(v_r-v3)[2] != (r_p0-r30)[2]*(v_r-v3)[1]: #prod form to avoid div by zero
                        break
                    print("partial find", t1, t2)
                else:
                    print("Found:", t1, t2, r_p0, v_r)

partial find 5 3
partial find 5 3
partial find 5 3
Found: 5 3 [24 13 10] [-3  1  2]


#### Pure geometric method
The following method would've been useful, and quick, if there were at least 2 coplanar lines. But it's not the case:

In [151]:
with open("input24.txt", "r") as f:
    data = [list(map(int,d.replace(" @ ",",").split(","))) for d in f.read().split("\n")]
for i in range(len(data)):
    x10, y10, z10, vx1, vy1, vz1 = data[i]
    for line2 in data[i+1:]:
        x20, y20, z20, vx2, vy2, vz2 = line2
        if vx1*vy2==vy1*vx2 and vy1*vz2==vz1*vy2 and vz1*vx2==vx1*vz2:
            print("Parallel lines:", data[i], line2)
        elif (vx1*vy2-vy1*vx2)*(vx1*(z10-z20) + vz1*(x20-x10)) == (vx1*vz2-vz1*vx2)*(vx1*(y10-y20) + vy1*(x20-x10)):
            print("Lines that meet:", data[i], line2)
#no lines are coplanar in the input!!!!

Yes, it only requires there being any 2 coplanar lines.

If at least 2 lines are coplanar, the solution becomes easier using the plane common to them: the rock line must belong to that common plane. Then check the intersection of that plane with each other hail line, and many points of the rock path will be known, in position and time. Condition not satisfied for the input, but so for the example input, which has 2 parallel lines.

In [8]:
with open("exampleinput24.txt", "r") as f:
    data = [list(map(int,d.replace(" @ ",",").split(","))) for d in f.read().split("\n")]
for i in range(len(data)):
    r10, v1 = np.array(data[i][:3]), np.array(data[i][3:])
    for line2 in data[i+1:]:
        r20, v2 = np.array(line2[:3]), np.array(line2[3:])
        if (np.cross(v1, v2)==0).all(): #v1 // v2
            print("Parallel lines:", data[i], line2)
            parallel_lines = [data[i], line2]
        elif (np.cross( np.cross(v1,v2), np.cross(v1, r20-r10) ) == 0).all()  : #require that v1, v2, and r20-r10 belong to the same plane
            print("Lines that meet:", data[i], line2)

Parallel lines: [18, 19, 22, -1, -1, -2] [20, 25, 34, -2, -2, -4]


From the geometry, that plane can be found using that both $\vec v_1$ and $\vec r_{20}-\vec r_{10}$ belong to it, so the normal is $\vec n := \vec v_1 \times (\vec r_{20}-\vec r_{10})$ (non-normalized normal haha). The final constant is adjusted demanding that $\vec r_{10}$ belongs to the plane, and the equation of the plane is
$$\vec n\cdot \vec r = \vec n \cdot \vec r_{10}$$
With that equation, the collision with only two hailstones (here subindexed as $3$ and $4$) suffices to find the initial position and velocity. The collision times are
$$ t_3 = \frac{\vec n \cdot (\vec r_{10} - \vec r_{30})}{\vec n\cdot \vec v_3}\, \ \ \ \ t_4 = \frac{\vec n \cdot (\vec r_{10} - \vec r_{40})}{\vec n\cdot \vec v_4}$$
<!-- : \ \mathrm{same, but} \ \ 3\to4$$-->
And the final equations are
$$ \vec v_p = \frac{\vec r_4(t_4)-\vec r_3(t_3)}{t_4-t_3} = \frac{\vec r_{40} - \vec r_{30} + \vec v_4t_4 - \vec v_3t_3}{t_4-t_3}$$
$$ \vec r_{p_0}= \vec r_{p}(t) - \vec v_pt = \vec r_{p}(t_3) - \vec v_pt_3 = \vec r_{30} + (\vec v_3-\vec v_p)t_3$$
<!-- $$[\vec v_1 \times (\vec r_{20}-\vec r_{10})]\cdot \vec r = [\vec v_1 \times (\vec r_{20}-\vec r_{10})]\cdot \vec r_{10} = \vec r_{10} \cdot (\vec v_1 \times \vec r_{20})$$ --> 

In [17]:
with open("exampleinput24.txt", "r") as f:
    data = [list(map(int,d.replace(" @ ",",").split(","))) for d in f.read().split("\n")]

def find_coplanar(data):
    for i in range(len(data)):
        r10, v1 = np.array(data[i][:3]), np.array(data[i][3:])
        for line2 in data[i+1:]:
            r20, v2 = np.array(line2[:3]), np.array(line2[3:])
            if (np.cross( np.cross(v1,v2), np.cross(v1, r20-r10) ) == 0).all()  : #require that v1, v2 and r20-r10 belong to the same plane
                coplanar_lines = [data[i], line2]
                return coplanar_lines #returns the first it finds
    raise ValueError("No coplanar lines found")

coplanar_lines = find_coplanar(data)
r10, v1 = np.array(coplanar_lines[0][:3]), np.array(coplanar_lines[0][3:])
r20, v2 = np.array(coplanar_lines[1][:3]), np.array(coplanar_lines[1][3:])

#normal vector
normal = np.cross(v1, r20-r10)
normal = (normal/np.gcd.reduce(normal)).astype(int)

#2 other vectors, can be changed if not the same as the parallel ones:
line3, line4 = data[3], data[4]
r30, v3 = np.array(line3[:3]), np.array(line3[3:])
r40, v4 = np.array(line4[:3]), np.array(line4[3:])

#collision times
t3, t4 = (normal@(r10-r30)/(normal@v3)).astype(int), (normal@(r10-r40)/(normal@v4)).astype(int)
#rock velocity and initial position:
vp, rp0 = ((r40-r30+v4*t4-v3*t3)/(t4-t3)).astype(int), r30 + (v3-vp)*t3
print("Initial position:" , rp0, "\b, initial velocity:", vp,"\b, output:",rp0.sum())

Initial position: [24 13 10], initial velocity: [-3  1  2], output: 47
